In [2]:
import sys, os
modules_path = os.path.join('..', '..')
if modules_path not in sys.path: sys.path.insert(0, modules_path)

from pprint import pformat, pp

from forgather.config import (
    preprocess_config,
    load_config,
    load_whitelist_as_set,
    materialize_config,
    enumerate_whitelist_exceptions,
    pconfig,
)

from forgather import Latent
from aiws.dotdict import DotDict

In [77]:
from typing import Callable, List, Any
import os

def build_rule(
    target: str | os.PathLike,
    recipe: Callable,
    prerequisites: List[str | os.PathLike],
    loader: Callable,
) -> Any:
    print('build_rule')

    assert isinstance(recipe, Callable)
    assert isinstance(loader, Callable)
    
    if os.path.exists(target):
        build_target = False
        target_mtime = os.path.getmtime(target)
        print(f"t {target} mtime: {target_mtime}")
        for dependency in prerequisites:
            dep_mtime = os.path.getmtime(dependency)
            print(f"d {dependency} mtime: {dep_mtime}")
            if target_mtime < dep_mtime:
                build_target = True
                break
    else:
        build_target = True
        
    if build_target:
        output = recipe()
        if output is not None:
            return output
    return loader()


In [71]:
import jinja2
from jinja2.sandbox import SandboxedEnvironment

def loader(file_name):
    print("load")
    with open(file_name, 'r') as f:
        return f.read()

def recipe(input_file, output_file):
    print("build")
    environment = SandboxedEnvironment(
        loader = jinja2.FileSystemLoader(searchpath='.'),
        auto_reload=True,
        undefined=jinja2.StrictUndefined,
    )
    
    template = environment.get_template(input_file)
    output = template.render()
    with open(output_file, 'w') as f:
        f.write(output)
    #return output

In [78]:
input_file = 'template.jinja'
output_file = 'rendered.txt'

latent_loader = Latent(loader, output_file, as_callable=True)
latent_builder = Latent(recipe, input_file, output_file, as_callable=True)
latent_builder = Latent(build_rule, target=output_file, recipe=latent_builder, loader=latent_loader, prerequisites=[input_file])

latent_builder()

build_rule
t rendered.txt mtime: 1720631053.2752779
d template.jinja mtime: 1720631050.1272845
load


'Foobarff'

In [85]:
config_src = """
-- set input_file = 'template.jinja'
-- set output_file = 'rendered.txt'

.define: &loader !callable:__main__:loader
        file_name: "{{ output_file }}"
        as_callable: True

.define: &builder !callable:__main__:recipe
    input_file: "{{ input_file }}"
    output_file: "{{ output_file }}"
    as_callable: True

rendered: !callable:__main__:build_rule
    target: "{{ output_file }}"
    recipe: *builder
    loader: *loader
    prerequisites: [ "{{ input_file }}" ]
"""

config = materialize_config(config_src, load_method='from_string')
pconfig(config)

build_rule
t rendered.txt mtime: 1720631053.2752779
d template.jinja mtime: 1720631050.1272845
load
config: rendered: 'Foobarff'
pp_config:
       1: .define: &loader !callable:__main__:loader
       2:         file_name: "rendered.txt"
       3:         as_callable: True
       4: 
       5: .define: &builder !callable:__main__:recipe
       6:     input_file: "template.jinja"
       7:     output_file: "rendered.txt"
       8:     as_callable: True
       9: 
      10: rendered: !callable:__main__:build_rule
      11:     target: "rendered.txt"
      12:     recipe: *builder
      13:     loader: *loader
      14:     prerequisites: [ "template.jinja" ]
  
pp_whitelist: None
whitelist: None


In [88]:
config_def = """
!callable:__main__:squared [2]
"""

def squared(x):
    return x**2

materialize_config(config_def, load_method='from_string').config

4